In [8]:
!pip install gdown


In [29]:
import gdown
import zipfile

# Descargar el archivo desde Google Drive
gdown.download("https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ", "dataset.zip", quiet=False)

# Extraer el contenido del ZIP
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()
    print("Dataset extraído exitosamente.")


Downloading...
From (original): https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ
From (redirected): https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ&confirm=t&uuid=54bd6049-8524-4936-aeb6-fef19b1835d1
To: /content/dataset.zip
100%|██████████| 363M/363M [00:10<00:00, 33.8MB/s]


Dataset extraído exitosamente.


In [37]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Add
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall, AUC

def is_directory_empty(directory):
    """Verifica si un directorio está vacío."""
    if not os.path.exists(directory):
        print(f"El directorio {directory} no existe.")
        return True
    return len(os.listdir(directory)) == 0

class SceneClassifier:
    def __init__(self, train_dir, test_dir, pred_dir, target_size=(150, 150), batch_size=32):
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.pred_dir = pred_dir
        self.target_size = target_size
        self.batch_size = batch_size
        self.models = {}
        self.history = {}

    def load_data(self):
      """Carga y preprocesa los datos."""
      train_datagen = ImageDataGenerator(rescale=1.0 / 255)
      test_datagen = ImageDataGenerator(rescale=1.0 / 255)

      # Cargar los datos de entrenamiento y prueba
      self.train_generator = train_datagen.flow_from_directory(
          self.train_dir, target_size=self.target_size, batch_size=self.batch_size, class_mode='categorical'
      )
      self.test_generator = test_datagen.flow_from_directory(
          self.test_dir, target_size=self.target_size, batch_size=self.batch_size, class_mode='categorical'
      )

      # Verificar si el directorio de predicción tiene imágenes
      if is_directory_empty(self.pred_dir):
          print("No hay datos de predicción disponibles.")
          self.prediction_generator = None
      else:
          self.prediction_generator = test_datagen.flow_from_directory(
              self.pred_dir, target_size=self.target_size, batch_size=self.batch_size, class_mode=None, shuffle=False
          )
      print("Datos cargados exitosamente.")



    def build_dense_model(self):
        """Construye un modelo denso."""
        model = Sequential([
            Flatten(input_shape=(*self.target_size, 3)),
            Dense(128, activation='relu'),
            Dense(6, activation='softmax')
        ])
        self._compile_model(model, 'dense')

    def build_cnn_model(self):
        """Construye un modelo CNN."""
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(*self.target_size, 3)),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(6, activation='softmax')
        ])
        self._compile_model(model, 'cnn')

    def build_residual_model(self):
        """Construye un modelo con bloque residual."""
        input_layer = Input(shape=(*self.target_size, 3))

        # Transformamos la entrada para que coincida con las dimensiones de salida
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

        # Ajustamos la dimensión del input_layer para que coincida con 'x' usando una convolución 1x1
        input_transformed = Conv2D(32, (1, 1), padding='same')(input_layer)

        # Realizamos la suma (residual connection)
        residual = Add()([input_transformed, x])

        # Resto del modelo
        x = Flatten()(residual)
        output_layer = Dense(6, activation='softmax')(x)

        model = Model(inputs=input_layer, outputs=output_layer)
        self._compile_model(model, 'residual')


    def build_transfer_model(self):
        """Construye un modelo con transfer learning usando VGG16."""
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*self.target_size, 3))
        base_model.trainable = False

        model = Sequential([
            base_model,
            Flatten(),
            Dense(128, activation='relu'),
            Dense(6, activation='softmax')
        ])
        self._compile_model(model, 'transfer')

    def _compile_model(self, model, name):
        """Compila el modelo con las métricas necesarias."""
        model.compile(optimizer='adam', loss='categorical_crossentropy',
                      metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])
        self.models[name] = model

    def train(self, model_name, epochs=5):
        """Entrena un modelo específico."""
        model = self.models[model_name]
        history = model.fit(self.train_generator, epochs=epochs, validation_data=self.test_generator)
        self.history[model_name] = history

    def evaluate(self, model_name):
        """Evalúa un modelo y muestra las métricas detalladas."""
        model = self.models[model_name]
        y_true = self.test_generator.classes
        y_pred = np.argmax(model.predict(self.test_generator), axis=1)

        print(f"Resultados para {model_name}:")
        print(classification_report(y_true, y_pred, target_names=self.test_generator.class_indices.keys()))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=self.test_generator.class_indices.keys())
        plt.show()


    def predict(self, model_name):
      """Realiza predicciones sobre el conjunto de predicción."""
      if self.prediction_generator is None:
          print("No se puede realizar predicción: No hay datos de predicción disponibles.")
          return

      model = self.models[model_name]
      predictions = model.predict(self.prediction_generator)
      predicted_classes = np.argmax(predictions, axis=1)

      print(f"Predicciones para {model_name}: {predicted_classes}")
      return predicted_classes


    def plot_history(self, model_name):
        """Grafica la pérdida y precisión."""
        history = self.history[model_name]

        plt.figure(figsize=(14, 5))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
        plt.plot(history.history['val_loss'], label='Pérdida Validación')
        plt.title(f'{model_name} - Pérdida')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
        plt.plot(history.history['val_accuracy'], label='Precisión Validación')
        plt.title(f'{model_name} - Precisión')
        plt.legend()

        plt.show()


In [ ]:
# Instancia del clasificador
classifier = SceneClassifier(
    train_dir='seg_train/seg_train',
    test_dir='seg_test/seg_test',
    pred_dir='seg_pred/seg_pred'
)

# Cargar los datos
classifier.load_data()

# Construir los modelos
classifier.build_dense_model()
classifier.build_cnn_model()
classifier.build_residual_model()
# classifier.build_transfer_model()

# Entrenar y evaluar los modelos
for model_name in classifier.models.keys():
    classifier.train(model_name, epochs=200)
    classifier.evaluate(model_name)
    classifier.plot_history(model_name)

# Realizar predicciones solo si hay datos disponibles
for model_name in classifier.models.keys():
    classifier.predict(model_name)



Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 0 images belonging to 0 classes.
Datos cargados exitosamente.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


439/439 ━━━━━━━━━━━━━━━━━━━━ 22s 44ms/step - accuracy: 0.3361 - auc: 0.6618 - loss: 8.6548 - precision: 0.3566 - recall: 0.3073 - val_accuracy: 0.4000 - val_auc: 0.7028 - val_loss: 2.4552 - val_precision: 0.4278 - val_recall: 0.3720
Epoch 2/200
439/439 ━━━━━━━━━━━━━━━━━━━━ 37s 37ms/step - accuracy: 0.4550 - auc: 0.7806 - loss: 1.6377 - precision: 0.5414 - recall: 0.3553 - val_accuracy: 0.3337 - val_auc: 0.6928 - val_loss: 1.5835 - val_precision: 0.6078 - val_recall: 0.1983
Epoch 3/200
439/439 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.3116 - auc: 0.6953 - loss: 1.5371 - precision: 0.7631 - recall: 0.1563 - val_accuracy: 0.3790 - val_auc: 0.7656 - val_loss: 1.4617 - val_precision: 0.6938 - val_recall: 0.1223
Epoch 4/200
439/439 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - accuracy: 0.4077 - auc: 0.7727 - loss: 1.4264 - precision: 0.7500 - recall: 0.1555 - val_accuracy: 0.3723 - val_auc: 0.7220 - val_loss: 1.5245 - val_precision: 0.8299 - val_recall: 0.0813
Epoch 5/200
439/439 ━━━━━━━━━━━━